In [13]:
%load_ext ipython_unittest
%load main.ipynb 


The ipython_unittest extension is already loaded. To reload it, use:
  %reload_ext ipython_unittest


NameError: name 'users' is not defined

In [12]:
%%unittest_main
class MyTest(unittest.TestCase):
    def test_1_plus_1_equals_2(self):
        sum = 1 + 1
        self.assertEqual(sum, 2)

    def test_2_plus_2_equals_4(self):
        self.assertEqual(2 + 2, 4)

Success

..
----------------------------------------------------------------------
Ran 2 tests in 0.000s

OK


<unittest.runner.TextTestResult run=2 errors=0 failures=0>